In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/Data Mining (CSE-4891)/Project files/dataset.csv'

In [3]:
import pandas as pd

df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')

In [14]:
import pandas as pd
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam

# Text cleaning function
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove digits
    text = text.strip().lower()          # Convert to lowercase
    return text

# Clean the Banglish text column
df['cleaned_banglish'] = df['banglish'].apply(clean_text)

# Label encoding for the 'label' column
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_banglish'], df['label_encoded'], test_size=0.2, random_state=42)

# Tokenize and Pad Sequences for BERT
tokenizer = BertTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

# Use padding and truncation for uniform sequence length
max_len = 128
train_sequences = tokenizer(X_train.tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors="tf")
test_sequences = tokenizer(X_test.tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors="tf")

# Convert labels to categorical format for classification
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Ensure inputs are tensors
input_ids_train = train_sequences['input_ids']
attention_mask_train = train_sequences['attention_mask']

input_ids_test = test_sequences['input_ids']
attention_mask_test = test_sequences['attention_mask']

from tensorflow.keras.layers import Input
from transformers import TFBertModel

# Define inputs properly using the Keras Input API
input_ids = Input(shape=(128,), dtype='int32', name='input_ids')
attention_mask = Input(shape=(128,), dtype='int32', name='attention_mask')

# Load BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Get BERT outputs
bert_output = bert_model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)

### Transfer Learning: Bangla BERT with LSTM Model ###
def build_bert_lstm_model():
    # Load the pre-trained BERT model
    bert_model = TFBertModel.from_pretrained('sagorsarker/bangla-bert-base')

    # Pass input tensors directly into BERT model
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

    # Extract BERT outputs (using last_hidden_state for token embeddings)
    bert_output = bert_model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
    bert_sequence_output = bert_output['last_hidden_state']

    # Add an LSTM layer on top of the BERT sequence output
    lstm_layer = Bidirectional(LSTM(128))(bert_sequence_output)

    # Dense layer for classification
    output = Dense(len(label_encoder.classes_), activation='softmax')(lstm_layer)

    # Define and compile the model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=Adam(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Build and train the model
bert_lstm_model = build_bert_lstm_model()

# Training the model
bert_lstm_model.fit(
    [input_ids_train, attention_mask_train],
    y_train_categorical,
    epochs=3,
    batch_size=32,
    validation_data=([input_ids_test, attention_mask_test], y_test_categorical)
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_5' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_5' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 128), dtype=int32, sparse=None, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 128), dtype=int32, sparse=None, name=attention_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=True
  • training=False

In [7]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam


# Text cleaning function
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove digits
    text = text.strip().lower()          # Convert to lowercase
    return text

# Clean the Banglish text column
df['cleaned_banglish'] = df['banglish'].apply(clean_text)

# Label encoding for the 'label' column
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_banglish'], df['label_encoded'], test_size=0.2, random_state=42)

# Tokenize and Pad Sequences for BERT
tokenizer = BertTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

# Use padding and truncation for uniform sequence length
max_len = 128
train_sequences = tokenizer(X_train.tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors="tf")
test_sequences = tokenizer(X_test.tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors="tf")

# Convert labels to categorical format for classification
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Ensure inputs are tensors
input_ids_train = tf.convert_to_tensor(train_sequences['input_ids'])
attention_mask_train = tf.convert_to_tensor(train_sequences['attention_mask'])

input_ids_test = tf.convert_to_tensor(test_sequences['input_ids'])
attention_mask_test = tf.convert_to_tensor(test_sequences['attention_mask'])

### Transfer Learning: Bangla BERT with LSTM Model ###
def build_bert_lstm_model():
    bert_model = TFBertModel.from_pretrained('sagorsarker/bangla-bert-base')

    # Define Input layers
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

    # Extract BERT outputs (using last_hidden_state for token embeddings)
    bert_output = bert_model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
    bert_sequence_output = bert_output['last_hidden_state']

    # Add an LSTM layer on top of the BERT sequence output
    lstm_layer = Bidirectional(LSTM(128))(bert_sequence_output)

    # Dense layer for classification
    output = Dense(len(label_encoder.classes_), activation='softmax')(lstm_layer)

    # Define and compile the model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=Adam(learning_rate=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Build and train the model
bert_lstm_model = build_bert_lstm_model()

# Training the model
bert_lstm_model.fit(
    [input_ids_train, attention_mask_train],
    y_train_categorical,
    epochs=3,
    batch_size=32,
    validation_data=([input_ids_test, attention_mask_test], y_test_categorical)
)



# Evaluate BERT+LSTM Model
y_pred_bert = np.argmax(bert_lstm_model.predict([test_sequences['input_ids'], test_sequences['attention_mask']]), axis=-1)
print("BERT+LSTM Classification Report:\n", classification_report(y_test, y_pred_bert, target_names=label_encoder.classes_))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are i

ValueError: Exception encountered when calling layer 'tf_bert_model_3' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_3' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 128), dtype=int32, sparse=None, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 128), dtype=int32, sparse=None, name=attention_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=True
  • training=False

In [ ]:
### Naive Bayes Classifier ###
# Convert text to bag-of-words for Naive Bayes
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Train a Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_counts, y_train)

# Predict and evaluate performance
y_pred_nb = nb_classifier.predict(X_test_counts)
print("Naive Bayes Classification Report:\n", classification_report(y_test, y_pred_nb, target_names=label_encoder.classes_))